# Web Usage Mining

## Dataset webuage

### Filter mengambil request url .html, status 200, dan request methode get

In [ ]:
import pandas as pd

# ===========================================
# 1. Baca file CSV
# ===========================================
file_path = "webuage.csv"
df = pd.read_csv(file_path, delimiter=';')

# ===========================================
# 2. Filter data sesuai kriteria
# ===========================================
filtered_df = df[
    (df["Request URI"].str.contains(".html", case=False, na=False)) &
    (df["Status"] == 200) &
    (df["Request method"].str.upper() == "GET")
].reset_index().rename(columns={"index": "Original Row"})

# ===========================================
# 3. Tampilkan hasil
# ===========================================
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)
pd.set_option('display.colheader_justify', 'center')

print(f"✅ Jumlah baris hasil filter: {len(filtered_df)}\n")
print(filtered_df.head(20).to_string(index=False))  # tampilkan 20 baris pertama

# ===========================================
# 4. (Opsional) Simpan ke file CSV
# ===========================================
filtered_df.to_csv("filtered_web_log.csv", index=False)


✅ Jumlah baris hasil filter: 75656

 Original Row  Remote host   Remote logname Remote user     Request time     Request method Request URI Request Protocol  Status  Size of response (incl. headers)
       0       65.55.147.227       -             -      2009-10-15T02:00:24Z      GET       /index.html     HTTP/1.1       200                  21878             
       1         65.55.86.34       -             -      2009-10-15T02:00:58Z      GET       /index.html     HTTP/1.1       200                   1416             
       2       148.188.55.88       -             -      2009-10-15T02:01:41Z      GET         /faq.html     HTTP/1.1       200                  10946             
       4      66.249.139.233       -             -      2009-10-15T02:02:09Z      GET         /faq.html     HTTP/1.1       200                  17247             
       5        72.30.50.248       -             -      2009-10-15T02:02:13Z      GET       /index.html     HTTP/1.0       200                   7883

### Merubah request time menjadi WIB

In [ ]:
# 2. Filter data sesuai kriteria
filtered_df = df[
    (df["Request URI"].str.contains(".html", case=False, na=False)) &
    (df["Status"] == 200) &
    (df["Request method"].str.upper() == "GET")
].reset_index().rename(columns={"index": "Original Row"})

# 3. Konversi waktu ke WIB (UTC+7)
filtered_df["Request time"] = pd.to_datetime(filtered_df["Request time"], utc=True, format='mixed', dayfirst=False) + pd.Timedelta(hours=7)
filtered_df["Request time"] = filtered_df["Request time"].dt.strftime("%Y-%m-%d %H:%M:%S") + " WIB"

# 4. Tampilkan hasil
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 150)

print(f"✅ Jumlah baris hasil filter: {len(filtered_df)}\n")
print(filtered_df.head(10).to_string(index=False))

# 5. (Opsional) Simpan ke file CSV
filtered_df.to_csv("filtered_web_log_wib.csv", index=False)

✅ Jumlah baris hasil filter: 75656

 Original Row  Remote host   Remote logname Remote user       Request time      Request method Request URI Request Protocol  Status  Size of response (incl. headers)
       0       65.55.147.227       -             -      2009-10-15 09:00:24 WIB      GET       /index.html     HTTP/1.1       200                  21878             
       1         65.55.86.34       -             -      2009-10-15 09:00:58 WIB      GET       /index.html     HTTP/1.1       200                   1416             
       2       148.188.55.88       -             -      2009-10-15 09:01:41 WIB      GET         /faq.html     HTTP/1.1       200                  10946             
       4      66.249.139.233       -             -      2009-10-15 09:02:09 WIB      GET         /faq.html     HTTP/1.1       200                  17247             
       5        72.30.50.248       -             -      2009-10-15 09:02:13 WIB      GET       /index.html     HTTP/1.0       200     

### Filter berdasarkan IP Address dan Request Protocol

In [ ]:
import pandas as pd

# --- Pastikan df dimuat dari file CSV ---
file_path = "webuage.csv"
df = pd.read_csv(file_path, delimiter=';')
# ----------------------------------------

# 2. Filter data sesuai kriteria
filtered_df = df[
    (df["Request URI"].str.contains(".html", case=False, na=False)) &
    (df["Status"] == 200) &
    (df["Request method"].str.upper() == "GET")
].reset_index().rename(columns={"index": "Original Row"})

# ----- SORT SESUAI PROTOCOL: HTTP/1.1 dulu, baru HTTP/1.0 -----

# Bersihkan protocol biar seragam
filtered_df["Request Protocol"] = (
    filtered_df["Request Protocol"]
    .astype(str)
    .str.strip()
    .str.upper()
)

# Mapping urutan sort
protocol_order = {"HTTP/1.1": 0, "HTTP/1.0": 1}

# Buat kolom sort sementara
filtered_df["protocol_sort"] = filtered_df["Request Protocol"].map(protocol_order)

# Urutkan
filtered_df = filtered_df.sort_values(
    by=["protocol_sort", "Original Row"],
    ascending=[True, True]
).drop(columns=["protocol_sort"])

# ---------------------------------------------------------------

# 3. Konversi waktu ke WIB (UTC+7)
filtered_df["Request time"] = (
    pd.to_datetime(filtered_df["Request time"], utc=True, format='mixed', dayfirst=False)
    + pd.Timedelta(hours=7)
)
filtered_df["Request time"] = filtered_df["Request time"].dt.strftime("%Y-%m-%d %H:%M:%S") + " WIB"

# 4. Tampilkan hasil
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 150)

print(f"✅ Jumlah baris hasil filter: {len(filtered_df)}\n")
print(filtered_df.head(10).to_string(index=False))

# 5. Simpan
filtered_df.to_csv("filtered_web_protocol.csv", index=False)

✅ Jumlah baris hasil filter: 75656

 Original Row    Remote host Remote logname Remote user            Request time Request method Request URI Request Protocol  Status  Size of response (incl. headers)
            0  65.55.147.227              -           - 2009-10-15 09:00:24 WIB            GET /index.html         HTTP/1.1     200                             21878
            1    65.55.86.34              -           - 2009-10-15 09:00:58 WIB            GET /index.html         HTTP/1.1     200                              1416
            2  148.188.55.88              -           - 2009-10-15 09:01:41 WIB            GET   /faq.html         HTTP/1.1     200                             10946
            4 66.249.139.233              -           - 2009-10-15 09:02:09 WIB            GET   /faq.html         HTTP/1.1     200                             17247
            8    65.55.80.97              -           - 2009-10-15 09:02:51 WIB            GET /index.html         HTTP/1.1     200   

## Dataset NASA

In [5]:
import pandas as pd

# ===========================================
# 1. Baca dataset NASA (misalnya nasa.csv)
# ===========================================
file_path = "nasa.csv"   # ganti sesuai nama file kamu
df = pd.read_csv(file_path, delimiter=";")   # sesuaikan delimiter kalau bukan ';'

# ===========================================
# 2. Tampilkan informasi dataset
# ===========================================

print("\n=== Jumlah Baris & Kolom ===")
print(df.shape)

print("\n=== 10 Baris Pertama dalam Format Tabel ===")
print(df.head(10).to_string(index=False))



=== Jumlah Baris & Kolom ===
(103763, 7)

=== 10 Baris Pertama dalam Format Tabel ===
 Unnamed: 0        host      time method                             url  response   bytes
          0 ***.novo.dk 805465029    GET                       /ksc.html     200.0  7067.0
          1 ***.novo.dk 805465031    GET      /images/ksclogo-medium.gif     200.0  5866.0
          2 ***.novo.dk 805465051    GET    /images/MOSAIC-logosmall.gif     200.0   363.0
          3 ***.novo.dk 805465053    GET       /images/USA-logosmall.gif     200.0   234.0
          4 ***.novo.dk 805465054    GET      /images/NASA-logosmall.gif     200.0   786.0
          5 ***.novo.dk 805465058    GET     /images/WORLD-logosmall.gif     200.0   669.0
          6 ***.novo.dk 805465068    GET /shuttle/missions/missions.html     200.0  8678.0
          7 ***.novo.dk 805465071    GET        /images/launchmedium.gif     200.0 11853.0
          8 ***.novo.dk 805465153    GET       /images/KSC-logosmall.gif     200.0  1204.0
   

### Filter url .html, response 200, methode GET, time di rubah ke wib

In [6]:
import pandas as pd

# Load
file_path = "nasa.csv"
df = pd.read_csv(file_path, delimiter=";", low_memory=False)

# Hilangkan spasi nama kolom
df.columns = df.columns.str.strip()

print(df.columns)   # cek

# Pastikan kolom 'time' benar-benar ada
if 'time' not in df.columns:
    raise ValueError(f"Kolom 'time' tidak ditemukan. Kolom yang ada: {df.columns.tolist()}")

# Convert epoch
df['time'] = pd.to_datetime(df['time'], unit='s')

# Filter
filtered_df = df[
    (df['url'].str.contains('.html', case=False, na=False)) &
    (df['response'] == 200) &
    (df['method'].str.upper() == 'GET')
].reset_index(drop=True)

# Print
print(filtered_df.head(10).to_string(index=False))

# Save
filtered_df.to_csv("filtered_nasa_wib.csv", index=False)
print("\n✅ File berhasil disimpan: filtered_nasa_wib.csv")


Index(['Unnamed: 0', 'host', 'time', 'method', 'url', 'response', 'bytes'], dtype='object')
 Unnamed: 0                    host                time method                                          url  response   bytes
          0             ***.novo.dk 1995-07-11 12:17:09    GET                                    /ksc.html       200  7067.0
          6             ***.novo.dk 1995-07-11 12:17:48    GET              /shuttle/missions/missions.html       200  8678.0
         12             ***.novo.dk 1995-07-11 12:23:01    GET    /shuttle/resources/orbiters/columbia.html       200  6922.0
         13             ***.novo.dk 1995-08-09 07:02:48    GET /shuttle/missions/sts-69/mission-sts-69.html       200 11264.0
         23             ***.novo.dk 1995-08-09 07:05:38    GET              /shuttle/countdown/liftoff.html       200  4665.0
         26             ***.novo.dk 1995-08-09 07:07:40    GET               /shuttle/countdown/lps/fr.html       200  1879.0
         29 001.msy4.commu

### Converter angka ke huruf

In [9]:
def num_to_letters(n):
    """0→A, 1→B, ..., 25→Z, 26→AA, dst."""
    result = ""
    n += 1
    while n > 0:
        n -= 1
        result = chr(65 + (n % 26)) + result
        n //= 26
    return result

### Menambahkan Session

In [10]:
def process_web_usage(df):
    """
    - Filter hanya URL .html
    - Membuat label huruf untuk setiap halaman
    - Membentuk sesi 20 menit per Host
    """

    # --- Filter .html ---
    df = df[df["url"].str.endswith(".html")].copy()

    # --- Buat mapping URL → huruf ---
    unique_urls = df["url"].unique()
    url_map = {url: num_to_letters(i) for i, url in enumerate(unique_urls)}
    df["URL_Label"] = df["url"].map(url_map)

    # --- Sortir ---
    df = df.sort_values(by=["Host", "Timestamp"])

    results = []

    # --- Proses sesi per host ---
    for host, group in df.groupby("Host"):
        group = group.sort_values("Timestamp").reset_index(drop=True)

        sessions = []
        cur_session = []
        prev_time = None

        for _, row in group.iterrows():
            ts = row["Timestamp"]
            label = row["URL_Label"]

            if prev_time is None:
                cur_session.append((ts, label))
            else:
                delta = (ts - prev_time).total_seconds() / 60

                # sesi baru jika >20 menit
                if delta > 20:
                    sessions.append(cur_session)
                    cur_session = [(ts, label)]
                else:
                    cur_session.append((ts, label))

            prev_time = ts

        # simpan sesi terakhir
        if cur_session:
            sessions.append(cur_session)

        results.append({
            "Host": host,
            "Sessions": sessions
        })

    return results, url_map, df

In [18]:
# Create a copy to avoid SettingWithCopyWarning if further modifications are made
processed_df = filtered_df.copy()

# Rename columns to match the 'process_web_usage' function's expectations
processed_df.rename(columns={'time': 'Timestamp', 'host': 'Host', 'url': 'url'}, inplace=True)

# Ensure Timestamp is datetime object for calculation in process_web_usage
processed_df['Timestamp'] = pd.to_datetime(processed_df['Timestamp'])

# Call the function to get the sessions and url_map
sessions_data, url_map, _ = process_web_usage(processed_df)

# Convert url_map to DataFrame
url_mapping_df = pd.DataFrame([
    {"URL": url, "Label": label}
    for url, label in url_map.items()
])

# ==============================
# Menyimpan
# ==============================

url_mapping_df.to_csv("url_mapping.csv", index=False)
print("✅ File CSV tersimpan sebagai: url_mapping.csv")

# Tampilkan hasil DataFrame
url_mapping_df


✅ File CSV tersimpan sebagai: url_mapping.csv


,URL,Label
0,/ksc.html,A
1,/shuttle/missions/missions.html,B
2,/shuttle/resources/orbiters/columbia.html,C
3,/shuttle/missions/sts-69/mission-sts-69.html,D
4,/shuttle/countdown/liftoff.html,E
...,...,...
107,/shuttle/missions/sts-70/woodpecker.html,DD
108,/shuttle/technology/sts-newsref/spacelab.html,DE
109,/persons/nasa-cm/jmd.html,DF
110,/history/apollo/apollo-4/apollo-4.html,DG


In [19]:
table_rows = []

for item in sessions_data:
    host = item["Host"]
    sessions = item["Sessions"]

    for session_idx, session in enumerate(sessions, start=1):
        for ts, label in session:
            table_rows.append({
                "Host": host,
                "Session": session_idx,
                "Timestamp": ts,
                "Page": label
            })

usage_table = pd.DataFrame(table_rows)

usage_table

,Host,Session,Timestamp,Page
0,***.novo.dk,1,1995-07-11 12:17:09,A
1,***.novo.dk,1,1995-07-11 12:17:48,B
2,***.novo.dk,1,1995-07-11 12:23:01,C
3,***.novo.dk,2,1995-08-09 07:02:48,D
4,***.novo.dk,2,1995-08-09 07:05:38,E
...,...,...,...,...
18544,129.105.106.27,1,1995-07-05 14:17:05,F
18545,129.105.106.27,1,1995-07-05 14:19:38,O
18546,129.105.107.224,1,1995-08-25 17:12:44,S
18547,129.105.107.224,1,1995-08-25 17:13:55,K


In [16]:
# 5. MEMBANGUN TABEL BINARY PER HALAMAN (A=1, lainnya=0)

def build_binary_table(sessions_data, url_map):
    all_labels = sorted(url_map.values())  # A, B, C, ...
    rows = []

    for host_data in sessions_data:
        host = host_data["Host"]

        for session in host_data["Sessions"]:
            for ts, label in session:

                # buat baris kosong (semua 0)
                row = {l: 0 for l in all_labels}

                # halaman yang dikunjungi = 1
                row[label] = 1

                # tambahkan kolom lain
                row["Time"] = ts.strftime("%H:%M")
                row["Host"] = host

                rows.append(row)

    return pd.DataFrame(rows)


binary_table = build_binary_table(sessions_data, url_map)

print("=== Tabel Binary Halaman Dikunjungi ===")
display(binary_table.head(20))

# Simpan CSV bila diperlukan
binary_table.to_csv("binary_web_usage.csv", index=False)

=== Tabel Binary Halaman Dikunjungi ===


,A,AA,AB,AC,AD,AE,AF,AG,AH,AI,...,S,T,U,V,W,X,Y,Z,Time,Host
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,12:17,***.novo.dk
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,12:17,***.novo.dk
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,12:23,***.novo.dk
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,07:02,***.novo.dk
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,07:05,***.novo.dk
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,07:07,***.novo.dk
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,06:55,001.msy4.communique.net
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,21:24,007.thegap.com
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,21:28,007.thegap.com
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,21:37,007.thegap.com
